# Ⅰ. 환경 셋팅

In [4]:
import pandas as pd
import kss
import openpyxl
import numpy as np
import re

# 데이터 인덱스 뷰 키우기
pd.set_option('display.max_row', 600)

# 모든 데이터 내용 다 보이게하기
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_colwidth', None)

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # This is added back by InteractiveShellApp.init_path()


# Ⅱ. 데이터 로드
- df : "한예슬is" 채널부터 500개
- df_all : 전체 데이터

In [5]:
# df30000 : 유튜브 댓글을 크롤링해온 raw 댓글 데이터
df = pd.read_csv("./data/df30000.csv")
df = df.iloc[2040:2540,:]

df_all = pd.read_csv("./data/df30000.csv")

# Ⅲ. 데이터 전처리

### 1) 문장 분리
    - kss.split_sentences() => 숫자/엔터/이모티콘 자동 분리 안 됨
    - 숫자 => 1. 2. 3. / 1) 2) 3) / Q1 Q2 / 그냥 숫자는 질문에 포함된 경우가 많음.

In [6]:
wb = openpyxl.Workbook()
sheet = wb.active
sheet.append(["댓글인덱스","추가일시", "전체 인덱스", "채널명", "채널인덱스", "글구분","댓글내용", "좋아요"])

In [47]:
n=0

for i in range(len(df_all['댓글내용'])):
    
    # kss 이용하여 문장 분리
    for j in kss.split_sentences(df_all.iloc[i,5]):
        
        # 숫자로 구분된 질문 split
        for f in re.split("[0-9]+\.|[0-9]+\)|Q[0-9]+",j):
            
            if f =="":
                continue
                
            sheet.append([n,df_all.iloc[i,0],df_all.iloc[i,1],df_all.iloc[i,2],df_all.iloc[i,3],df_all.iloc[i,4],f.strip(),df_all.iloc[i,6]])
            n+=1

# 문장 분리된 데이터 엑셀에 저장
wb.save("0716complete_v5.csv")

### 2) 질문인 문장 뽑아내기
데이터는 분리된 데이터로 사용

- 질문이 아닌 문장

- 질문인 문장
    - ?가 들어간 문장들 다 질문인지 확인해보기

In [11]:
df_s = pd.read_csv("./data/0716complete_v5.csv")
df_sf = df_s

In [12]:
# \n, 자세히 보기 삭제
df_s['댓글내용']=df_s['댓글내용'].str.replace("자세히 보기","")
df_s['댓글내용']=df_s['댓글내용'].str.replace("\\n","")

# 질문 데이터에서 공통적으로 발견되는 단어들로 질문데이터 추출
df_sd = df_s.loc[(-df_s['댓글내용'].str.contains("\?"))| (df_s['댓글내용'].str.contains("궁금") )| (df_s['댓글내용'].str.contains("어떻게") )| (df_s['댓글내용'].str.contains("알려") )| (df_s['댓글내용'].str.contains("나요") )| (df_s['댓글내용'].str.contains("가요") )]

# 한글이 발견되지 않는 데이터 없애기
pattern1 = re.compile('[가-힣]+')
same = pd.DataFrame({"궁금":df_s['댓글내용'],"없애고궁금":df_sd['댓글내용']})
same['영어없앰'] = df_sd['댓글내용'].str.findall(pattern1)
same['영어없앤 문장'] = same['없애고궁금'].map(lambda x : ' '.join(pattern1.findall(str(x))))
same['영어댓글삭제'] = same['없애고궁금'][same['영어없앤 문장']!=""]

# 특수문자, 자음 등 삭제
same['기호없앰'] = same['영어댓글삭제'].map(lambda x : re.sub('[-=+;,_#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》(ㄱ-ㅎ)+(ㅏ-ㅣ)+]', '', str(x)))

In [13]:
#nan 데이터 삭제
df_sf['댓글내용'] = same['기호없앰']
df_sf = df_sf[df_sf['댓글내용'].str.contains('nan') == False]

# 너무 짧거나 너무 긴 데이터 삭제
df_sf = df_sf[(df_sf['댓글내용'].str.len()>4)&(df_sf['댓글내용'].str.len()<150)]

# Ⅳ. Tokenization

In [14]:
from konlpy.tag import Okt
okt = Okt()

### 명사만 추출

In [16]:
def get_tokens(x):
    n_list = [i for i in okt.nouns(x) if len(i) > 1] if x else []
    return list(set(n_list))

In [17]:
df_sf['명사'] = df_sf['댓글내용'].map(get_tokens)

In [19]:
# 명사개수가 2이상인것만
df_sfn= df_sf[df_sf['명사'].map(len)>1]

# Ⅴ. 워드 임베딩

In [21]:
from gensim.models import Word2Vec
# word2vec대신 deeplearning4j 도 있음
model = Word2Vec.load('embedding.save')

unable to import 'smart_open.gcs', disabling that module


- 단어의 벡터 = 한 문장의 각 성분을 벡터화
- 문장의 벡터 = 단어 벡터의 평균

In [30]:
def get_sentence_mean_vector(morphs):
    vector = []
    for i in morphs:
        try:
            vector.append(model.wv[i])
        except KeyError as e:
            pass
    try:
        return np.mean(vector, axis=0,dtype='float64')
    except IndexError as e:
        pass

In [31]:
# wv 컬럼에 문장 벡터 저장
df_sfn['wv'] = df_sfn['명사'].map(get_sentence_mean_vector)

C:\Users\hp\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\hp\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
df_sfn['wv']

0                         [-0.20564180612564087, 0.07338651642203331, 0.02586244884878397, -0.03859498677775264, -0.19431698322296143, 0.09358405321836472, 0.15972375683486462, 0.07096372358500957, -0.18971075722947717, 0.10147631168365479, -0.06263351067900658, -0.24972780421376228, 0.13013158086687326, -0.010757612995803356, -0.14799673296511173, 0.04990845546126366, -0.05056224371219287, -0.029607398435473442, 0.2302249250933528, -0.07123731821775436, 0.07508966512978077, 0.029368510469794273, 0.12176048319088295, -0.23362693469971418, -0.0009452719241380692, -0.020262273028492928, 0.020517578348517418, -0.2036267053335905, -0.07627824787050486, -0.26219476014375687, 0.054718961007893085, -0.030328338965773582, -0.03994962479919195, -0.06630613654851913, 0.28534314036369324, -0.1589092633803375, 0.005167549476027489, 0.12003187090158463, 0.04830338107421994, -0.05074120871722698, 0.21284273359924555, 0.20018185768276453, 0.028543520718812943, 0.051716760732233524, -0.126525142230093

In [33]:
# null 데이터 삭제
df_sfn_del = df_sfn[df_sfn['wv'].isnull()==False]

-----

# Ⅵ. 클러스터링

### 한예슬 채널

In [34]:
hy = df_sfn_del[df_sfn_del['채널명']=="한예슬 is"]

In [35]:
# Gaussian 사용
from sklearn.mixture import GaussianMixture
import time

word_vectors = hy['wv'].to_list() 
# 집단 개수 100ro
num_clusters = 100

Gaussian = GaussianMixture(n_components = num_clusters, covariance_type = 'full')
idx = Gaussian.fit_predict(word_vectors)
hy['category'] = idx

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


### 차원 축소

In [39]:
# 차원 축소
from sklearn.manifold import TSNE

X = hy['wv'].to_list()
y = hy['category'].to_list()

import os.path
import pickle

tsne_filepath = 'tsne3001.pkl'

# File Cache
if not os.path.exists(tsne_filepath):
    tsne = TSNE(random_state=42)
    tsne_points = tsne.fit_transform(X)
    with open(tsne_filepath, 'wb+') as f:
        pickle.dump(tsne_points, f)
else: # Cache Hits!
    with open(tsne_filepath, 'rb') as f:
        tsne_points = pickle.load(f)

tsne_df = pd.DataFrame(tsne_points, index=range(len(X)), columns=['x_coord', 'y_coord'])
tsne_df['user_bio'] = hy['댓글내용'].to_list()
tsne_df['cluster_no'] = y

# Ⅶ. 시각화

In [40]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
from bokeh.palettes import brewer

output_notebook()

Loading BokehJS ...

In [41]:
# 100개로 분류하므로 10 * 10
colors = brewer["Spectral"][10]*10

colormap = {tsne_df['cluster_no'].unique()[i]: colors[i] for i in range(len(tsne_df['cluster_no'].unique()))}


colors = [colormap[x] for x in tsne_df['cluster_no']]

tsne_df['color'] = colors


# Bokeh Datasouce 만들기
plot_data = ColumnDataSource(
    data=tsne_df.to_dict(orient='list')
)

# Plot 만들기(배경)
tsne_plot = figure(
    title='YouTube Q&A clustering',
    plot_width = 650,
    plot_height = 650,
    active_scroll='wheel_zoom',
    output_backend="webgl",
)

# 해당 Hover 툴팁 만들기
tsne_plot.add_tools(
    HoverTool(
        tooltips='@user_bio'
    )
)

tsne_plot.circle(
    source=plot_data,
    x='x_coord',
    y='y_coord',
    line_alpha=0.3, 
    fill_alpha=0.2,
    size=10,
    fill_color='color',
    line_color='color',
)

GlyphRenderer(id='1041', ...)

In [42]:
# 각 값들 추가해주기 
tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

show(tsne_plot)

In [44]:
tsne_df[['user_bio','cluster_no']].sort_values(by='cluster_no').head(600)

,user_bio,cluster_no
514,Q 함께 작업했던 남자배우중 최고의 매너남은 콕 찍어주세요,0
396,예슬이언니가 콧소리없이 얘기하면 어떤느낌인가요,1
522,언니에게 삶이란 어떤 의미에요,1
85,언니 mbti 뭔가요,1
469,얼굴이 그렇게 작으면 어떤느낌인가요,1
236,언니 첫사랑 이별을 주제로 한 이야기가 궁금해용 ♀,1
115,언니도 남들보다 뒤처진다는 느낌을 받기도 하시나용미모 열일하시구 코로나 조심해용,1
112,이팅사운드 말고 언니 목소리로,1
336,글로벌한 친구들이 있을만한 계기가 있을 것 같은데 그게 넘넘 궁금해요,2
335,언니는 유럽에도 다니고 미국에 친구도있는것 같고 글로벌하게 친구들이 있는것 같은데 이게 미국과 한국만 오가도 그게 가능한거에요,2


In [62]:
tsne_df[['user_bio','cluster_no']].sort_values(by='cluster_no').to_excel("clustergau.xlsx")